# Тестовое задание Avito.

Далее использованы представления данных и модели, показавшие лучший результат на эксперименте (https://github.com/aapiskotin/MachineLearningProjects/blob/master/avito_clf/avito_classification_research.ipynb)

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [2]:
category = pd.read_csv('data/category.csv', index_col='category_id')

In [3]:
category.head()

,name
category_id,
0,Бытовая электроника|Телефоны|iPhone
1,Бытовая электроника|Ноутбуки
2,Бытовая электроника|Телефоны|Samsung
3,Бытовая электроника|Планшеты и электронные кни...
4,"Бытовая электроника|Игры, приставки и программ..."


In [4]:
data_train = pd.read_csv('data/train.csv', index_col='item_id')

In [5]:
data_train.head()

,title,description,price,category_id
item_id,,,,
0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [6]:
data_test = pd.read_csv('data/test.csv', index_col='item_id')

# EDA

## Выделение признаков из текста
- считаем количество латинских букв
- считаем длину
- избавляемся от знаков препинания
- Все встречающиеся слова приводим к нормальной форме
- кодируем в вектора через CountVectorizer

In [8]:
from multiprocessing import Pool

In [9]:
import re

english_check = re.compile(r'[a-zA-Z]')

def count_eng(text):
    counter = 0
    for c in text:
        if english_check.match(c):
            counter += 1
            
    return counter

In [10]:
with Pool(processes=4) as pool:
    data_train['title_eng_count'] = pool.map(count_eng, data_train['title'])
    data_test['title_eng_count'] = pool.map(count_eng, data_test['title'])

    data_train['descr_eng_count'] = pool.map(count_eng, data_train['description'])
    data_test['descr_eng_count'] = data_test['description'].apply(count_eng)
    pool.terminate()

In [11]:
data_train['title_len'] = data_train['title'].apply(len)

data_train['descr_len'] = data_train['description'].apply(len)

In [12]:
import pymorphy2 as morphy
import string

morpher = morphy.MorphAnalyzer()

In [13]:
from functools import lru_cache

@lru_cache(maxsize=100000)
def get_normal_form (word):
    return morpher.normal_forms(word)[0]

In [14]:
def text_normalizer(text):
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower()
    words = text.split()
    normalized_text = ''
    for word in words:
        normalized_text += get_normal_form(word) + ' '
        
    return normalized_text.rstrip()

In [15]:
with Pool(processes=4) as pool:
    data_train['title_norm'] = pool.map(text_normalizer, data_train.title)
    data_train['desct_norm'] = pool.map(text_normalizer, data_train.description)
    
    data_test['title_norm'] = pool.map(text_normalizer, data_test.title)
    data_test['desct_norm'] = pool.map(text_normalizer, data_test.description)
    pool.terminate()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
title_vectorizer = CountVectorizer(binary=True)
title_features_train = title_vectorizer.fit_transform(data_train.title)
title_features_test = title_vectorizer.transform(data_test.title)

descr_vectorizer = CountVectorizer(binary=True)
description_features_train = descr_vectorizer.fit_transform(data_train.description)
description_features_test = descr_vectorizer.transform(data_test.description)

## Числовые признаки
Отшкалируем числовые признаки

In [18]:
num_features_columns = ['price', 'title_eng_count', 'descr_eng_count', 'title_len', 'descr_len']

In [19]:
from sklearn.preprocessing import StandardScaler

In [57]:
scaler = StandardScaler()

num_features_scaled_train = scaler.fit_transform(data_train.loc[:, num_features_columns])
num_features_scaled_test = scaler.transform(data_test.loc[:, num_features_columns])

/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversio

In [61]:
num_features_scaled_test = np.nan_to_num(num_features_scaled_test)

## Обучение модели
В качестве модели лучший результат показала логистическая регрессия

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [63]:
features = sp.hstack((title_features_train, description_features_train, num_features_scaled_train))
features_test = sp.hstack((title_features_test, description_features_test, num_features_scaled_test))

In [23]:
X_train, X_val, y_train, y_val = train_test_split(features, data_train['category_id'], 
                                         random_state=648, test_size=0.25, shuffle=True)

In [24]:
logreg = LogisticRegression(n_jobs=2, penalty='l2')

In [25]:
logreg.fit(X_train, y_train)
y_hat = logreg.predict(X_val)
print(str(logreg))
print('Accuracy: ' + str(accuracy_score(y_val, y_hat)))

/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=2,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy: 0.8806095767282236


Получили хороший скор. Теперь подсчитаем accuracy для каждого уровня иерархии

Для начала выделим все подкатегории

In [26]:
sub_cats_disordered = category['name'].str.split(pat='|', expand=True)

In [27]:
sub_cats_disordered.head()

,0,1,2,3
category_id,,,,
0,Бытовая электроника,Телефоны,iPhone,None
1,Бытовая электроника,Ноутбуки,None,None
2,Бытовая электроника,Телефоны,Samsung,None
3,Бытовая электроника,Планшеты и электронные книги,Планшеты,None
4,Бытовая электроника,"Игры, приставки и программы",Игровые приставки,None


In [28]:
sub_cats = sub_cats_disordered.copy()
for i in range(1, len(sub_cats.columns)):
    cur_col = sub_cats_disordered[0].copy()
    for j in range(1, i + 1):
        cur_col += '|' + sub_cats_disordered[j].apply(str)
    sub_cats[i] = cur_col

In [29]:
sub_cats_coded = sub_cats.copy()

In [30]:
for col in sub_cats.columns:
    cat_code = dict(zip(sub_cats[col].unique(), range(len(sub_cats[col].unique()))))
    sub_cats_coded[col] = sub_cats_coded[col].apply(lambda x: cat_code[x])

In [31]:
sub_cats_coded.tail()

,0,1,2,3
category_id,,,,
49,3,20,46,49
50,3,21,47,50
51,3,17,48,51
52,3,21,49,52
53,3,22,50,53


In [32]:
def accuracy(y_true, y_pred):
    return np.sum(y_pred == y_true) / len(y_true)

### Accuracy по категориям

In [33]:
for sub_cat_level in sub_cats_coded.columns:
    translate = np.vectorize(lambda x: sub_cats_coded.iloc[x, sub_cat_level])
    print("Accuracy for level " + str(sub_cat_level) + ': ' + str(accuracy(translate(y_val), translate(y_hat))))

Accuracy for level 0: 0.9583265239418206
Accuracy for level 1: 0.9386255924170616
Accuracy for level 2: 0.8846461840169962
Accuracy for level 3: 0.8806095767282236


Теперь переучиваем модель на всей выборке данных и составляем предикт

In [34]:
logreg.fit(features, data_train['category_id'])

/Users/aapiskotin/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=2,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
y_pred_test = logreg.predict(features_test)

pd.DataFrame({'item_id': data_test.index, 'category_id':y_pred_test}).to_csv('the_prediction.csv', index=None)